## Tracking the COVID-19 outbreak

This is a companion notebook for the dashboard web application.

It provides transparency on the code used to source, prepare and visualise the data.

Although the app1.py file contains all of this, the majority of the code in that file is specific to defining the set-up, styling and dynamic behaviour of the web application itself. 

This notebook is relevant to audiences interested in the content itself.

For more information on building interactive data-driven dashboards in Python (and only Python, no JavaScript here!), I highly recommend looking at <a href=https://plotly.com/dash/> Plotly Dash </a> 

The dashboard is accessible at http://covid-ww.herokuapp.com/


**Note on streaming vs manual data sourcing**

Earlier versions of this app streamed data direct from the relevant APIs. This meant each call to the website would triggered a refresh of 7 distinct data calls, resulting in longer load-time and frequent request time-out errors. 

Since each dataset is refreshed once every 24 hours, I decided to store static versions of all the relevant data locally, and perform a parallel job each morning to refresh all of the data at once

In [1]:
#Import necessary libraries

import pandas as pd
import numpy as np
import plotly.graph_objs as go
import requests
import datetime as dt
from plotly.subplots import make_subplots
import io

In [2]:
# Set up chart layouts

l_map=go.Layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    geo={
    'visible': True, 
    'resolution':50, 
    'showcountries':True, 
    'countrycolor':"grey",
    'showsubunits':True, 
    'subunitcolor':"White",
    'showframe':False,
    'coastlinecolor':"slategrey",
    'countrycolor':'white',
    }
)

#UK Map
l_mapbox=go.Layout(
  margin={"r":0,"t":0,"l":0,"b":0},
  mapbox_style="carto-positron",
  mapbox={
    'bearing':0,
    'center':go.layout.mapbox.Center(lat=55.3781,lon=-3.6360),
    'pitch':0,
    'zoom':4
    },
  
 )

#HEATMAP & LINE CHART
l_trend=go.Layout(
  margin={"r":0,"t":30,"l":0,"b":0},
  template="plotly_white",
  yaxis={"tickfont":{"size":10}},
  xaxis={"tickfont":{"size":10}},
  legend={'x':0.01, 'y':0.98, 'font':{'size':10}}
)

#SIMPLE BARS
l_bar_s=go.Layout(
  height=180,
  margin={"r":0,"t":0,"l":0,"b":0},
  template="plotly_white",
  yaxis={"tickfont":{"size":9}},
  xaxis={"tickfont":{"size":9}},
  legend={'x':0.02, 'y':0.96, 'font':{'size':9}}
  )

### Worldwide statistics

**Data Sources**

 - **Cases and deaths** timeseries data: <a href=https://github.com/CSSEGISandData> Johns Hopkins Universtity Centre for Systems Science and Engineering </a> 
 - **World Population by Country**:  <a href=https://population.un.org/wpp/Download/Standard/CSV/> UN World Population Prospects 2019 </a> 
 - **Lat/Long co-ordinates** for world countries: <a href=https://developers.google.com/public-data/docs/canonical/countries_csv> Google </a> 

**Import of cases, deaths and world population data**

In [3]:
#import timeseries data for cases (url_1) and deaths (url_2)
url_1='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
url_2='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
df1=pd.read_csv(url_1)
df2=pd.read_csv(url_2)

In [4]:
#Read helper files
ctrs=pd.read_csv('countries.csv').set_index("name")
continents=pd.read_csv("continents.csv").drop(columns="Unnamed: 0")
df_p=pd.read_csv("worldpop.csv").set_index("Location")

FileNotFoundError: [Errno 2] File countries.csv does not exist: 'countries.csv'

In [ ]:
#Define data transformations 

def prep_world_data(df):
    df=df.groupby(by="Country/Region").sum()
    df.drop(labels=["Lat", "Long"], axis=1, inplace=True)
    df.loc[:,:str(df.columns[-1])]=np.clip(df.loc[:,:str(df.columns[-1])], a_min=0, a_max=None)
    df.rename(index={"US": 'United States of America'}, inplace=True)
    return df

def prep_countries_data(d1,d2):
    d1=prep_world_data(d1)
    d2=prep_world_data(d2)
    countries=ctrs.merge(d1[d1.columns[-1]], left_index=True, right_index=True)
    countries.rename(columns={d1.columns[-1]: "Confirmed"}, inplace=True)
    countries=countries.merge(d2[d2.columns[-1]], left_index=True, right_index=True)
    countries.rename(columns={d2.columns[-1]: "Deaths"}, inplace=True)
    countries=countries.merge(df_p["PopTotal"], left_index=True, right_index=True)
    countries["Conf100"]=round(countries["Confirmed"]*100/countries["PopTotal"])
    countries["Deaths100"]=round(countries["Deaths"]*100/countries["PopTotal"])
    countries["Mortality"]=round(countries["Deaths"]*100/countries["Confirmed"],2)
    countries["text"]= countries.index +'<br>' + "Cases: " + countries["Confirmed"].astype('str') + '<br>' + "Deaths: "+ countries["Deaths"].astype('str')
    countries["text100"]= countries.index +'<br>' + "Cases per 100k pop.: " + countries["Conf100"].astype('str') + '<br>' + "Deaths per 100k pop.: "+ countries["Deaths100"].astype('str')
    return countries

def prep_world_trend(d):
    d=prep_world_data(d)
    world_trend=d.merge(continents, left_index=True, right_on="Country").groupby("Continent").sum()
    return world_trend

def prep_world_newcases(d):
    df=prep_world_trend(d)
    world_newcases=pd.DataFrame(index=df.index,columns=list(df.columns[1:]))
    for i in range(len(df.columns)-1):
        f=df[df.columns[i+1]]-df[df.columns[i]]
        world_newcases[world_newcases.columns[i]]=f
    return world_newcases

def prep_world_capita (d):
    d1=prep_world_data(d)
    t=d1.merge(df_p, left_index=True, right_index=True)
    a=t.loc[:, :d1.columns[-1]].values.transpose()
    b=100/t["PopTotal"].values
    c=a*b
    t.loc[:, :d1.columns[-1]]=c.transpose()
    return t

def prep_rolling_avg(d):
    d1=prep_world_capita(d)
    d11=prep_world_data(d)
    sr_5d=pd.DataFrame(index=d1.index,columns=list(d11.columns[5:]))
    for i in range(len(d11.columns)-5):
        f=(d1[d1.columns[i+5]]-d1[d1.columns[i]])/5
        sr_5d[sr_5d.columns[i]]=f
    return sr_5d

In [ ]:
#Define helper charts (RHS OF WORLD MAP)

#Country top 15's
def make_fig_2_3(d1,d2,text):
    fig = go.Figure(go.Bar(y=prep_countries_data(d1,d2).sort_values(by=[text], ascending=True).index[-15:],
                          x=prep_countries_data(d1,d2).sort_values(by=[text],ascending=True)[text][-15:],
                         orientation='h'
                        )
                 )
    fig.update_layout(l_bar_s)
    return fig

#Cumulative cases/deaths by continent

def make_fig_4(d):
    fig = go.Figure(data=[go.Bar(x=pd.to_datetime(np.array(prep_world_trend(d).columns)),
                          y=prep_world_trend(d).iloc[0],
                          name=prep_world_trend(d).index[0]),
                        go.Bar(x=pd.to_datetime(np.array(prep_world_trend(d).columns)),
                          y=prep_world_trend(d).iloc[1],
                          name=prep_world_trend(d).index[1]),
                        go.Bar(x=pd.to_datetime(np.array(prep_world_trend(d).columns)),
                          y=prep_world_trend(d).iloc[2],
                          name=prep_world_trend(d).index[2]),
                        go.Bar(x=pd.to_datetime(np.array(prep_world_trend(d).columns)),
                          y=prep_world_trend(d).iloc[3],
                          name=prep_world_trend(d).index[3]),
                        go.Bar(x=pd.to_datetime(np.array(prep_world_trend(d).columns)),
                          y=prep_world_trend(d).iloc[4],
                          name=prep_world_trend(d).index[4]),
                        go.Bar(x=pd.to_datetime(np.array(prep_world_trend(d).columns)),
                          y=prep_world_trend(d).iloc[5],
                          name=prep_world_trend(d).index[5])]
                        )
    fig.update_layout(barmode='stack')
    fig.update_layout(l_bar_s)
    return fig


#New cases/deaths by continent
def make_fig_5(d):
    fig = go.Figure(data=[go.Bar(x=pd.to_datetime(np.array(prep_world_newcases(d).columns)),
                          y=prep_world_newcases(d).iloc[0],
                          name=prep_world_newcases(d).index[0]),
                        go.Bar(x=pd.to_datetime(np.array(prep_world_newcases(d).columns)),
                          y=prep_world_newcases(d).iloc[1],
                          name=prep_world_newcases(d).index[1]),
                        go.Bar(x=pd.to_datetime(np.array(prep_world_newcases(d).columns)),
                          y=prep_world_newcases(d).iloc[2],
                          name=prep_world_newcases(d).index[2]),
                        go.Bar(x=pd.to_datetime(np.array(prep_world_newcases(d).columns)),
                          y=prep_world_newcases(d).iloc[3],
                          name=prep_world_newcases(d).index[3]),
                        go.Bar(x=pd.to_datetime(np.array(prep_world_newcases(d).columns)),
                          y=prep_world_newcases(d).iloc[4],
                          name=prep_world_newcases(d).index[4]),
                        go.Bar(x=pd.to_datetime(np.array(prep_world_newcases(d).columns)),
                          y=prep_world_newcases(d).iloc[5],
                          name=prep_world_newcases(d).index[5]),
                        ]
                        )
    fig.update_layout(barmode='stack')
    fig.update_layout(l_bar_s)
    return fig

**World map and Worst-hit countries**

In [ ]:
d = go.Figure(go.Scattergeo(
    lon=prep_countries_data(df1,df2)["longitude"],
    lat=prep_countries_data(df1,df2)["latitude"],
    text = prep_countries_data(df1,df2)['text100'],
    hoverinfo = 'text',
    marker=dict(
        size= 0.5*prep_countries_data(df1,df2)["Conf100"],
        line_width=0.5,
        sizemode='area'
    )))
fig1=go.Figure(data=d)
fig1.update_layout(l_map)
fig1.update_geos(projection_type="natural earth", lataxis_showgrid=False, lonaxis_showgrid=False)
fig1.show()

In [ ]:
make_fig_2_3(df1,df2,"Confirmed")

In [ ]:
make_fig_2_3(df1,df2,"Conf100")

In [ ]:
make_fig_4(df1)

In [ ]:
make_fig_5(df1)

In [ ]:
li=["Switzerland", "United Kingdom", "Spain", "Italy", "France"]
d3=[{
    'x': pd.to_datetime(np.array(prep_world_data(df1).columns)),
    'y': prep_world_data(df1)[prep_world_data(df1).index==country].values[0],
    'name': country
    } for country in li]
fig3=go.Figure(data=d3,layout=l_trend)
#fig3.update_layout(yaxis_type="log")
fig3.show()        

In [ ]:
data=go.Heatmap(
        z=prep_rolling_avg(df2).loc[li],
        x=pd.to_datetime(np.array(prep_rolling_avg(df2).columns)),
        y=li,
        colorscale='Blues',
        colorbar={"thickness":10, "tickfont":{"size":10}},
        ) 
fig4=go.Figure(data=data,layout=l_trend)
fig4.show()

### USA statistics

**Data Sources**

- **Cases and Deaths**: <a href=https://github.com/CSSEGISandData> Johns Hopkins Universtity Centre for Systems Science and Engineering </a> 
- **Population by County**: <a href=https://www.census.gov/programs-surveys/popest/data/data-sets.html> United States Census Bureau </a>
- **County co-ordinates, Regional state groupings**: Same source as Cases and Deaths

**Data ingestion & tranformation**

In [ ]:
url_3='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
url_4='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
url_5='https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv'

df3=pd.read_csv(url_3)
df4=pd.read_csv(url_4).drop("Population", axis=1)
us_regions=pd.read_csv(url_5)
us_pop=pd.read_csv('us_pop.csv')

def prep_county_data(d):
    df=d.drop(labels=["UID","iso2", "iso3","Province_State","Country_Region"], axis=1)
    df.drop(df[df["Admin2"].isnull()].index, inplace=True)
    df.drop(df[df["FIPS"].isnull()].index, inplace=True)
    return df   

def prep_state_data(d):
    df=d.groupby(by="Province_State").sum()
    df.drop(labels=["UID", "code3", "FIPS", "Lat", "Long_"], axis=1, inplace=True)
    df.loc[:,:str(df.columns[-1])]=np.clip(df.loc[:,:str(df.columns[-1])], a_min=0, a_max=None)
    return df

def prep_county_sum(d1,d2):
    dd1=prep_county_data(d1)
    dd2=prep_county_data(d2)
    df=dd1[["Admin2", "FIPS", "Lat", "Long_", dd1.columns[-1]]]
    df=df.rename(columns={dd1.columns[-1]: "Confirmed"})
    df=df.merge(dd2[["FIPS",dd2.columns[-1]]], on="FIPS")
    df=df.rename(columns={dd2.columns[-1]: "Deaths"})
    df=df.merge(us_pop[["FIPS", "STNAME","CTYNAME", "POPESTIMATE2019", "Abbreviation"]], on="FIPS")
    df["Mortality"]=round(df["Deaths"]*100/df["Confirmed"],2)
    df["Conf100"]=df["Confirmed"]*100000/df["POPESTIMATE2019"]
    df["Deaths100"]=df["Deaths"]*100000/df["POPESTIMATE2019"]
    df["Ctylabel"]=df["Admin2"]+ ", "+df["Abbreviation"]
    df["text"]= df["Ctylabel"] + '<br>Cases: ' + (round(df['Confirmed'],1).astype(str))+ '<br>Deaths: ' + (round(df['Deaths'],1).astype(str))
    df["text100"]= df["Ctylabel"] + '<br>Cases per 100k pop: ' + (round(df['Conf100'],1).astype(str))+ '<br>Deaths per 100k pop: ' + (round(df['Deaths100'],1).astype(str))
    return df

def prep_state_sum(d1,d2):
    df=prep_county_sum(d1,d2)
    df=df[["STNAME", "Confirmed", "Deaths", "POPESTIMATE2019"]].groupby("STNAME").sum()
    df["Conf100"]=df["Confirmed"]*100000/df["POPESTIMATE2019"]
    df["Deaths100"]=df["Deaths"]*100000/df["POPESTIMATE2019"]
    df["Mortality"]=round(df["Deaths"]*100/df["Confirmed"],2)
    return df

def prep_us_trend(d):
    df=prep_state_data(d)
    df=df.merge(us_regions, left_index=True, right_on="State")
    df=df.groupby("Region").sum()
    return df

def prep_us_newcases(d):
    df=prep_us_trend(d)
    dff=pd.DataFrame(index=df.index,columns=list(df.columns[1:]))
    for i in range(len(df.columns)-1):
        f=df[df.columns[i+1]]-df[df.columns[i]]
        dff[dff.columns[i]]=f
    return dff


In [ ]:
#County total cases
def make_fig_7_8(d1,d2,text):
    fig = go.Figure(go.Bar(y=prep_county_sum(d1,d2).sort_values(by=[text], ascending=True)["Ctylabel"][-15:],
                          x=prep_county_sum(d1,d2).sort_values(by=[text],ascending=True)[text][-15:],
                         orientation='h'
                        )
                 )
    fig.update_layout(l_bar_s)
    return fig

#Region cululative cases

def make_fig_9(d):
    fig = go.Figure(data=[go.Bar(x=pd.to_datetime(np.array(prep_us_trend(d).columns)),
                          y=prep_us_trend(d).iloc[0],
                          name=prep_us_trend(d).index[0]),
                        go.Bar(x=pd.to_datetime(np.array(prep_us_trend(d).columns)),
                          y=prep_us_trend(d).iloc[1],
                          name=prep_us_trend(d).index[1]),
                        go.Bar(x=pd.to_datetime(np.array(prep_us_trend(d).columns)),
                          y=prep_us_trend(d).iloc[2],
                          name=prep_us_trend(d).index[2]),
                        go.Bar(x=pd.to_datetime(np.array(prep_us_trend(d).columns)),
                          y=prep_us_trend(d).iloc[3],
                          name=prep_us_trend(d).index[3])]

                         #orientation='h'
                        )
    fig.update_layout(barmode='stack')
    fig.update_layout(l_bar_s)
    return fig

#New cases by region

def make_fig_10(d):
    fig = go.Figure(data=[go.Bar(x=pd.to_datetime(np.array(prep_us_newcases(d).columns)),
                          y=prep_us_newcases(d).iloc[0],
                          name=prep_us_newcases(d).index[0]),
                        go.Bar(x=pd.to_datetime(np.array(prep_us_newcases(d).columns)),
                          y=prep_us_newcases(d).iloc[1],
                          name=prep_us_newcases(d).index[1]),
                        go.Bar(x=pd.to_datetime(np.array(prep_us_newcases(d).columns)),
                          y=prep_us_newcases(d).iloc[2],
                          name=prep_us_newcases(d).index[2]),
                        go.Bar(x=pd.to_datetime(np.array(prep_us_newcases(d).columns)),
                          y=prep_us_newcases(d).iloc[3],
                          name=prep_us_newcases(d).index[3])]
                         #orientation='h'
                        )
    fig.update_layout(barmode='stack')
    fig.update_layout(l_bar_s)
    return fig


In [ ]:
d= go.Scattergeo(
        lon = prep_county_sum(df3,df4)['Long_'],
        lat = prep_county_sum(df3,df4)['Lat'],
        text = prep_county_sum(df3,df4)['text'],
        hoverinfo = 'text',
        marker = dict(
                size = 0.05*(prep_county_sum(df3,df4)['Confirmed']),
                line_width=0.5,
                sizemode = 'area'
            ))


fig6=go.Figure(data=d)
fig6.update_layout(l_map)
fig6.update_geos(scope="usa",
            lataxis_showgrid=False, 
            lonaxis_showgrid=False
            )
fig6.show()

In [ ]:
make_fig_7_8(df3,df4, "Confirmed")

In [ ]:
make_fig_7_8(df3,df4, "Conf100")

In [ ]:
make_fig_9(df3)

In [ ]:
make_fig_10(df3)

### UK Zoom

**Data Sources**

 - **Cases**: 
 
     - England:   <a href=https://coronavirus.data.gov.uk/ > Public Health England </a>
     - Wales:  <a href=https://public.tableau.com/profile/public.health.wales.health.protection#!/vizhome/RapidCOVID-19virology-Public/Headlinesummary > Public Health Wales Coronavirus Dashboard</a>  
     - Scotland:  <a href=https://www.gov.scot/coronavirus-covid-19/ > Scottish Government</a> 
 - **Population**: <a href=https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/>  UK Office of National Statistics </a>
 - **Co-ordinates of local authorities**: <a href= http://geoportal.statistics.gov.uk/> UK Office of National Statistics Open Geography Portal </a>

**Data ingestion & transformation**

In [ ]:
url_6 = "https://coronavirus.data.gov.uk/downloads/csv/coronavirus-cases_latest.csv"
url_7='http://www2.nphs.wales.nhs.uk:8080/CommunitySurveillanceDocs.nsf/61c1e930f9121fd080256f2a004937ed/77fdb9a33544aee88025855100300cab/$FILE/Rapid%20COVID-19%20surveillance%20data.xlsx'
url_8='https://www.gov.scot/binaries/content/documents/govscot/publications/statistics/2020/04/trends-in-number-of-people-in-hospital-with-confirmed-or-suspected-covid-19/documents/covid-19-data-by-nhs-board/covid-19-data-by-nhs-board/govscot%3Adocument/HSCA%2B-%2BSG%2BWebsite%2B-%2BIndicator%2BTrends%2Bfor%2Bdaily%2Bdata%2Bpublication%2B-%2BHealth%2BBoard%2BBreakdown.xlsx?forceDownload=true'

uk_map=pd.read_csv("uk_map.csv").drop(columns="Unnamed: 0").groupby(by="ReportingArea").mean()
uk_pop=pd.read_csv("uk_pop.csv").drop(columns="Unnamed: 0").groupby(by="ReportingArea").sum()

def convert(val):
    step1=str(val)
    step2 = step1.replace(',','')
    step3 = step2.replace('**','')
    step4= int(step3)
    return step4

def prep_england_cases(url):
    s = requests.get(url).content
    england_cases = pd.read_csv(io.StringIO(s.decode('utf-8')))
    return england_cases

def prep_wales_cases(url):
    wales_cases=pd.read_excel (url, 'Tests by specimen date')
    return wales_cases

def prep_scot_cases(url):
    df=pd.read_excel(url, 'Table 1 - Cumulative cases',skiprows=2).replace({'*':0})
    df.rename(columns={
        'NHS Ayrshire & Arran':'Ayrshire & Arran', 'NHS Borders':'Borders',
           'NHS Dumfries & Galloway':'Dumfries and Galloway', 'NHS Fife':'Fife', 'NHS Forth Valley':'Forth Valley',
           'NHS Grampian':'Grampian', 'NHS Greater Glasgow & Clyde':'Greater Glasgow and Clyde', 'NHS Highland':'Highland',
           'NHS Lanarkshire':'Lanarkshire', 'NHS Lothian':'Lothian', 'NHS Orkney':'Orkney', 'NHS Shetland':'Shetland',
           'NHS Tayside':'Tayside', 'NHS Western Isles':'Eileanan Siar (Western Isles)'    
    }, inplace=True)
    scotland_cases=pd.DataFrame({
                       "ReportingArea":df.iloc[-1][1:-1].index.values, 
                       "Confirmed": df.iloc[-1][1:-1]})
    return scotland_cases

def prep_uk_scatter(url1,url2,url3):
    d1=prep_england_cases(url1)
    d2=prep_wales_cases(url2)
    d3=prep_scot_cases(url3)
    dd1=d1.drop(d1[d1["Area type"]=="Region"].index)
    dd1=dd1[["Area name", "Specimen date", "Cumulative lab-confirmed cases"]][dd1["Specimen date"]==dd1["Specimen date"].max()]
    dd1=dd1.drop(labels=["Specimen date"], axis=1)
    dd1.rename(columns={"Area name":"ReportingArea", "Cumulative lab-confirmed cases": "Confirmed"}, inplace=True)
    dd2=d2[d2["Specimen date"]==d2["Specimen date"].max()][["Local Authority", "Cumulative cases"]].rename(columns={"Local Authority": "ReportingArea", "Cumulative cases": "Confirmed"})
    df=dd1.append(dd2, ignore_index=True).append(d3, ignore_index=True)
    df=df.astype('str')
    df.set_index("ReportingArea", inplace=True)
    df=df.merge(uk_pop, right_index=True, left_index=True).merge(uk_map, on="ReportingArea")
    df["Confirmed"]=df["Confirmed"].apply(convert)
    df["Conf100"]=round(df["Confirmed"]*100000/df["All ages"])
    df["text"]= df.index+ ', '+ '<br>Cases: ' + (df['Confirmed']).astype(str)
    df["text100"]= df.index + ', '+ '<br>Cases per 100k pop: ' + (round(df['Conf100'],1).astype(str))
    return df

def prep_eng_trend(url):
    df=prep_england_cases(url)
    df=df[["Area name", "Specimen date", "Daily lab-confirmed cases", "Cumulative lab-confirmed cases"]]
    df=df.rename(columns={"Area name": "ReportingArea", "Daily lab-confirmed cases":"New cases", "Cumulative lab-confirmed cases": "Cumulative cases"})
    return df
    
def prep_wales_trend(url):
    df=prep_wales_cases(url)
    df=df[["Local Authority", "Specimen date", "Cases (new)", "Cumulative cases"]]
    df=df.rename(columns={"Local Authority": "ReportingArea", "Cases (new)":"New cases"})
    return df

def prep_scot_trend(url):
    d=df=pd.read_excel(url, 'Table 1 - Cumulative cases',skiprows=2).replace({'*':0})
    df=d[["Date", "Scotland"]].rename(columns={"Date": "Specimen date", "Scotland": "Cumulative cases"})
    return df

In [ ]:
#Worst hit local areas
def make_fig_12_13(url1,url2,url3,text):
    fig = go.Figure(go.Bar(y=prep_uk_scatter(url1,url2, url3).sort_values(by=[text], ascending=True).index[-15:],
                          x=prep_uk_scatter(url1,url2, url3).sort_values(by=[text],ascending=True)[text][-15:],
                         orientation='h'
                        )
                 )
    fig.update_layout(l_bar_s)
    return fig

def make_fig_14(url1,url2,url3):
    fig = go.Figure(data=[go.Bar(x=prep_eng_trend(url1)[prep_eng_trend(url1)["ReportingArea"]=="England"]["Specimen date"],
                          y=prep_eng_trend(url1)[prep_eng_trend(url1)["ReportingArea"]=="England"]["Cumulative cases"],
                          name="England"),
                        go.Bar(x=prep_wales_trend(url2).groupby("Specimen date").sum().index,
                          y=prep_wales_trend(url2).groupby("Specimen date").sum()["Cumulative cases"],
                          name="Wales"),
                        go.Bar(x=prep_scot_trend(url3)["Specimen date"],
                          y=prep_scot_trend(url3)["Cumulative cases"],
                          name="Scotland")]
                        )
    fig.update_layout(barmode='stack')
    fig.update_layout(l_bar_s)
    return fig

def make_fig_15(url1,url2,url3):
    fig = go.Figure(data=[go.Bar(x=prep_eng_trend(url1)[prep_eng_trend(url1)["ReportingArea"]=="England"]["Specimen date"],
                          y=prep_eng_trend(url1)[prep_eng_trend(url1)["ReportingArea"]=="England"]["New cases"],
                          name="England"),
                        go.Bar(x=prep_wales_trend(url2).groupby("Specimen date").sum().index,
                          y=prep_wales_trend(url2).groupby("Specimen date").sum()["New cases"],
                          name="Wales"),
                        go.Bar(x=prep_scot_trend(url3)["Specimen date"][1:],
                          y=prep_scot_trend(url3)["Cumulative cases"][1:].values - prep_scot_trend(url3)["Cumulative cases"][:-1].values,
                          name="Scotland")]
                        )
    fig.update_layout(barmode='stack')
    fig.update_layout(l_bar_s)
    return fig

In [ ]:
d=go.Scattermapbox(
    lon = prep_uk_scatter(url_6,url_7, url_8)['long'],
    lat = prep_uk_scatter(url_6,url_7, url_8)['lat'],
    text = prep_uk_scatter(url_6,url_7, url_8)['text100'],
    hoverinfo = 'text',
    marker = dict(
            size = 5*np.sqrt(prep_uk_scatter(url_6,url_7, url_8)['Conf100']),
            sizemode = 'area',
        symbol = 'circle'
        ))
fig11=go.Figure(data=d)
fig11.update_layout(l_mapbox)
fig11.show()

In [ ]:
make_fig_12_13(url_6, url_7, url_8, "Confirmed")

In [ ]:
make_fig_12_13(url_6, url_7, url_8, "Conf100")

In [ ]:
make_fig_14(url_6, url_7, url_8)

In [ ]:
make_fig_15(url_6, url_7, url_8)